# Outliers **removal** and **imputation** 

**Input**:
* DHIS2 **routine data** (formatted and aligned as per ush).
    * From Dataset **SNT_DHIS2_FORMATTED** as `XXX_routine.parquet` 
* Table of outliers: table containing only values flagged as outliers, based on 1 outliers detection method. This input is a choice of the user (see parameters)
    * From Dataset **DHIS2_OUTLIERS_DETECTION** as `XXX_outlier_<method_name>.parquet`

**Output**:
* DHIS2 **routine data _without_ outliers**: outliers are simply removed.
    * To Dataset **DHIS2_OUTLIERS_REMOVAL_IMPUTATION** as `XXX_routine_removed-outlier-<method_name>.parquet` (e.g., `COD_routine_outlier_mean3sd_removed.parquet`)
* DHIS2 **routine data** _without_ outliers &  **_with_ imputed values**: values flagged as outliers are replaced by imputed values.
    * To Dataset **DHIS2_OUTLIERS_REMOVAL_IMPUTATION** as `XXX_routine_removed-outlier-<method_name>_imputed.parquet`
    * Imputation uses the moving avergae as per:
  ```
  rollapply(IMPUTATION, # zoo::rollapply()
                               width = 3, 
                               FUN = function(x) ceiling(mean(x, na.rm = TRUE)), 
                               fill = NA, 
                               align = "center")
  ```

-------------------------------

**Parameters**:
* **How to treat outliers**:
    * remove
    * remove & replace with impuated values
<br>
<br>
* **Choice of outliers detection method**:
    * mean _n_ * SD
    * median _n_ * MAD
    * _n_ * IQR
    * Magic Glasses "Partial": file called `*_outlier_magic_glasses_partial_*` (MAD15 >> MAD10)
    * Magic Glasses "Complete": file called `*_outlier_magic_glasses_complete_*` (MAD15 >> MAD10 >> seasonal5 >> seasonal3)

---------------------------

## 0. Parameters

👇 these are now ⚡**pipeline parameters**⚡!

In [ ]:
# OUTLIER_METHOD <- "mean3sd"

#### Set Default values **if _not_ provided by pipeline**
This makes the execution flexible and "safe": nb can be run manually from here or be executed via pipeline, without having to change anything in the code!

🚨 **Important**: must use the `if (!exists("PARAMETER_NAME")) { ... }` formula to avoid over writing the parameters injected by Papermill (into special cell a top of nb, see OUTPUT nb)

In [ ]:
# Set BACKUP VALUE: name of config file to use
if (!exists("CONFIG_FILE_NAME")) {
  CONFIG_FILE_NAME <- "SNT_config.json"  # Default if not provided by pipeline
}

# Set BACKUP VALUE: 
if (!exists("OUTLIER_METHOD")) {
  OUTLIER_METHOD <- "median3mad"  # other options: "mean3sd", "iqr1-5", "magic_glasses_partial", "magic_glasses_complete" 
}                              # ⚠️ TO DO: in pipeline.py, make this choice dynamic based on files available in Dataset (subset part of filename that indicates the method
                               

## 1. Setup

### 1.1. Paths

In [ ]:
# Set BACKUP VALUE: root path - NEVER CHANGE THIS!
if (!exists("ROOT_PATH")) {
  ROOT_PATH <- "/home/hexa/workspace"  
}

In [ ]:
# PROJECT PATHS

# Project folders
CODE_PATH <- file.path(ROOT_PATH, 'code') # this is where we store snt_functions.r and snt_utils.r
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration') # .json config file
DATA_PATH <- file.path(ROOT_PATH, 'data') # same as in Datasets but /data/ gets over written every time a new version of Datasets is pushed

print(CODE_PATH)

### 1.2. Utils functions

In [ ]:
source(file.path(CODE_PATH, "snt_utils.r"))

### 1.3. Packages

In [ ]:
# List required pcks 
required_packages <- c("arrow", # for .parquet
                       "tidyverse",
                       "stringi", 
                       # "sf",
                       # "forecast",
                       "zoo",
                       "jsonlite", 
                       "httr", 
                       "reticulate")

# Execute function
install_and_load(required_packages)

#### Set environment to load openhexa.sdk from the right path

In [ ]:
# Set environment to load openhexa.sdk from the right path
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

### 1.4. Load and check `config` file

In [ ]:
# Load SNT config

config_json <- tryCatch({
        fromJSON(file.path(CONFIG_PATH, CONFIG_FILE_NAME)) # "SNT_config.json"
    },
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, CONFIG_FILE_NAME)) 
log_msg(msg)

#### **Checks for SNT mandatory configuration fields**

In [ ]:
# CHECK SNT configuration 
snt_config_mandatory <- c("COUNTRY_CODE", "DHIS2_ADMINISTRATION_1", "DHIS2_ADMINISTRATION_2") 

for (conf in snt_config_mandatory) {
    print(paste(conf, ":", config_json$SNT_CONFIG[conf]))
    if (is.null(config_json$SNT_CONFIG[[conf]])) {
        msg <- paste("Missing configuration input:", conf)
        cat(msg)   
        stop(msg)
    }
}

#### **Save config fields as variables**

In [ ]:
# Generic
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
ADMIN_1 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_1)
ADMIN_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)

# Which (aggregated) indicators to use to evaluate "activity" of an HF - for Reporting Rate method "ANY"
DHIS2_INDICATORS <- names(config_json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS)

In [ ]:
DHIS2_INDICATORS

In [ ]:
# Fixed routine formatting columns
# must keep&use `OU` as it contains unique ids (OU_NAME has homonimous values!)
fixed_cols <- c('PERIOD', 'YEAR', 'MONTH', 'ADM1_ID', 'ADM2_ID', 'OU_ID') # keep ADMX_ID only! 

print(paste("Fixed routine data ('dhis2_routine') columns (always expected): ", paste(fixed_cols, collapse=", ")))

## 2. Load Data

### 2.1. **Routine** data (DHIS2) 
already formatted & aggregated<br>
(output of pipeline "XXX" and stored in Dataset "**SNT_DHIS2_FORMATTED**")

In [ ]:
# DHIS2 Dataset extract identifier
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

# Load file from dataset
dhis2_routine <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_routine.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 routine data file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 routine data loaded from dataset : ", dataset_name, " dataframe dimensions: ", paste(dim(dhis2_routine), collapse=", "))
log_msg(msg)

In [ ]:
head(dhis2_routine, 3)

#### Checks on routine data cols

**fixed_cols**

In [ ]:
# Check if all "fixed" cols are present in dhis2_routine

actual_cols <- colnames(dhis2_routine) # dhis2_routine
missing_cols <- setdiff(fixed_cols, actual_cols) # Columns in fixed_cols but not in actual_cols)

# Check if all required columns are present
all_present <- length(missing_cols) == 0

if (all_present) { 
  log_msg(paste0("Success: The 'dhis2_routine' tibble contains all the expected 'fixed' columns: ",
                paste(fixed_cols, collapse = ", "), "."))
} else {
    log_msg(paste0(
      "🛑 Missing Columns: The following required columns are NOT present in 'dhis2_routine': ",
      paste(missing_cols, collapse = ", "),
      "."
    ))
  }

**DHIS2_INDICATORS**

In [ ]:
# Check if all "DHIS2_INDICATORS" cols are present in dhis2_routine

actual_cols <- colnames(dhis2_routine) # dhis2_routine
missing_cols <- setdiff(DHIS2_INDICATORS, actual_cols) # Columns in fixed_cols but not in actual_cols)
all_present <- length(missing_cols) == 0

if (all_present) { 
  log_msg(paste0("Success: The 'dhis2_routine' tibble contains all the expected 'DHIS2_INDICATORS' columns: ",
                paste(fixed_cols, collapse = ", "), "."))
} else {
    log_msg(paste0(
      "🚨 Missing Columns: The following columns for DHIS2 INDICATORS are NOT present in 'dhis2_routine': ",
      paste(missing_cols, collapse = ", "),
      ". 🚨 Looks like the ", CONFIG_FILE_NAME, " file was modified after extraction. 🚨 The analysis will continue WITHOUT the missing indicators."
    )
                   )
  }

### 2.2. Ouliers table

In [ ]:
# DHIS2 Dataset extract identifier
dataset_name_outliers <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_OUTLIERS_DETECTION

# Load file from dataset
outliers <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name_outliers, paste0(COUNTRY_CODE, "_outlier_", OUTLIER_METHOD, ".parquet")) }, # COD_outlier_iqr1-5.parquet
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 routine data file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("Table with outlier values loaded from dataset : ", dataset_name_outliers, " dataframe dimensions: ", paste(dim(outliers), collapse=", "))
log_msg(msg)

In [ ]:
print(dim(outliers))
head(outliers, 3)

# 3. Remove Outliers

### Pivot routine data 
Make long

In [ ]:
# Pivot `indicator_cols` = all columns except fixed columns
# indicator_cols <- colnames(dhis2_routine)[!(names(dhis2_routine) %in% fixed_cols)] # problem: includes `OU_NAME`
indicator_cols <- names(config_json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS) # instead, just extract list of indicators colsnames based on config file

# pivot routine data
dhis2_routine_long <- dhis2_routine %>%
    pivot_longer(#cols = all_of(indicator_cols),
                 cols = any_of(indicator_cols),  # ⚠️ TEMP switch as config.json was changed but not extracted data (some cols are missing) ⚠️
                 names_to = "INDICATOR",
                 values_to = "VALUE") 

In [ ]:
print(dim(dhis2_routine_long))
head(dhis2_routine_long,3)

### Format the **outliers** table: rename col `OUTLIER_<method_name>` to `OUTLIER`
🚨 Necessary because the **col flagging** the **outlier** values has a **different name** for each outlier detetcion method output <br>
(💡 consider changing this upstream ... )

In [ ]:
outliers <- outliers %>%
  rename_with(~ "OUTLIER", starts_with("OUTLIER_"))

In [ ]:
names(outliers)

### Join outliers on routine data
This adds the col `OUTLIER_*` to the (pivoted long) routine data.
Here, `NA`s are assigned to values that are not outliers. These `NA`s need to be replaced by `FALSE`

In [ ]:
# Join
# merged_data <- left_join(dhis2_routine_long, outliers, 
#                          by = c(fixed_cols, "INDICATOR", "VALUE")) 

dhis2_routine_long_outliers <- left_join(dhis2_routine_long, outliers, 
                                         by = c(fixed_cols, "INDICATOR", "VALUE")) 

# In col `OUTLIER_*`, replace `NA`s with `FALSE`
# merged_data <- merged_data %>%
# mutate(OUTLIER = if_else(is.na(OUTLIER), FALSE, OUTLIER))

dhis2_routine_long_outliers <- dhis2_routine_long_outliers %>%
mutate(OUTLIER = if_else(is.na(OUTLIER), FALSE, OUTLIER))

print(dim(dhis2_routine_long_outliers))
head(dhis2_routine_long_outliers, 3)

### Remove outliers
This was not in the original/older code (BFA). Added now as option: just remove all values flagged as outliers.
<br>
Later: **Export** as `XXX_routine_outlier-<method_name>_removed`.csv/parquet 

#### **Remove** outliers & pivot wider

In [ ]:
# was `routine_removed_outliers` now `dhis2_routine_long_outliers_removed`
dhis2_routine_long_outliers_removed <- dhis2_routine_long_outliers %>%
filter(OUTLIER == FALSE)

In [ ]:
print(dim(dhis2_routine_long_outliers_removed))
head(dhis2_routine_long_outliers_removed, 3)

In [ ]:
# When removing outliers, how many values are removed (= nr of values converted to NA)
nr_of_outliers <- dhis2_routine_long_outliers %>% filter(OUTLIER == TRUE) %>% nrow()
nr_of_values <- dhis2_routine_long_outliers %>% nrow()
perc_outliers <- round(nr_of_outliers/nr_of_values * 100, 2)

msg <- paste0(
  "Using outliers detection method *", OUTLIER_METHOD,
  "*. Identified and removed ", nr_of_outliers,
  " outliers (", perc_outliers, "% of values)."
)

# Log the message
if (!is.null(openhexa$current_run)) {
  openhexa$current_run$log_info(msg)
}

# log_msg(msg) # Also print to console - this makes it printed twice in the pipeline log

In [ ]:
dhis2_routine_outliers_removed <- dhis2_routine_long_outliers_removed %>%
select(all_of(fixed_cols), INDICATOR, VALUE) %>% # names() #  'PERIOD''YEAR''MONTH''ADM1_ID''ADM2_ID''OU''INDICATOR''VALUE'
pivot_wider(names_from = "INDICATOR", values_from = "VALUE") 

In [ ]:
print(dim(dhis2_routine_outliers_removed))
head(dhis2_routine_outliers_removed, 3)

# 4. Impute values: replace outliers based on rolling average

### Create col `TO_IMPUTE` 
Create `TO_IMPUTE` col from `VALUE`, and set to `NA` if value is flagged as outlier (namely, when `OUTLIER == TRUE`)

In [ ]:
dhis2_routine_long_outliers_imputed <- dhis2_routine_long_outliers %>%
  mutate(TO_IMPUTE = VALUE,
         TO_IMPUTE = if_else(OUTLIER == TRUE, NA_real_, TO_IMPUTE)) 

head(dhis2_routine_long_outliers_imputed, 3)

In [ ]:
# Sanity check
dhis2_routine_long_outliers_imputed %>% filter(OUTLIER == TRUE) %>% head(., 3)

### Impute: Calculate rolling average

In [ ]:
dhis2_routine_long_outliers_imputed <- dhis2_routine_long_outliers_imputed %>%
arrange(ADM1_ID, ADM2_ID, OU_ID, INDICATOR, PERIOD) %>%  # Ensure proper order
    group_by(ADM1_ID, ADM2_ID, OU_ID, INDICATOR) %>%  # Group data
    mutate(
        MOVING_AVG = zoo::rollapply(TO_IMPUTE, 
                                    width = 3, 
                                    FUN = function(x) ceiling(mean(x, na.rm = TRUE)), 
                                    fill = NA, 
                                    align = "center"),
        VALUE_IMPUTED = ifelse(is.na(TO_IMPUTE), MOVING_AVG, TO_IMPUTE)  # Replace NA with MOVING_AVG
    ) %>%  
    ungroup()


In [ ]:
print(dim(dhis2_routine_long_outliers_imputed))
head(dhis2_routine_long_outliers_imputed, 3)

In [ ]:
# # Curiosity check: see where NaN's introduced by `zoo::rollapply()`
# merged_data_imputation_imputed %>% filter(is.nan(VALUE_IMPUTED)) %>% head(., 3)

In [ ]:
# Set all `NaN`s to `NA`s
dhis2_routine_long_outliers_imputed$VALUE_IMPUTED[is.nan(dhis2_routine_long_outliers_imputed$VALUE_IMPUTED)] <- NA

### Replace values, and pivot wider
More specifically:
* replace outlier values with imputed values: simply rename `VALUE_IMPUTED` col to `VALUE`
* pivot table to wider (return to original structure)

In [ ]:
dhis2_routine_imputed <- dhis2_routine_long_outliers_imputed %>%
    mutate(VALUE = VALUE_IMPUTED) %>% # IMPUTATION_VALUE
    select(all_of(fixed_cols), INDICATOR, VALUE) %>% # names() #  'PERIOD''YEAR''MONTH''OU''ADM1_ID''ADM2_ID''INDICATOR''VALUE'
    pivot_wider(names_from = "INDICATOR", values_from = "VALUE") 

#### Sanity check: make sure imputing does not alter the number of rows in the table

In [ ]:
# Check dimensions and log error if they don't match
if (!isTRUE(all.equal(nrow(dhis2_routine), nrow(dhis2_routine_imputed)))) {
  msg_error <- paste0(
    "Error: Nr of rows in 'dhis2_routine' (",
    paste(dim(dhis2_routine), collapse = "x"),
    ") does not match nr of rows in 'dhis2_routine_imputed' (",
    paste(dim(dhis2_routine_imputed), collapse = "x"),
    ")."
  )

  if (!is.null(openhexa$current_run)) {
    openhexa$current_run$log_error(msg_error)
  }

  stop(msg_error) # stop the script and print the error message
}

# 5. Export Output tables
Export tables as .csv and .parquet files to `/data/dhis2_outliers_removal_imputation/` folder, then code in pipeline.py will handle the writing to Dataset 

In [ ]:
export_data <- function(data_object, file_suffix, file_type) {
  # Define the output directory
  output_dir <- file.path(DATA_PATH, "dhis2_outliers_removal_imputation")

  # Create the directory if it doesn't exist
  if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE)
  }

  # Construct the file name
  file_name <- paste0(COUNTRY_CODE, "_routine_outliers-", OUTLIER_METHOD, file_suffix, ".", file_type)

  # Construct the full path
  full_path <- file.path(output_dir, file_name)

  # Export the data based on file type
  if (file_type == "csv") {
    write_csv(data_object, full_path)
  } else if (file_type == "parquet") {
    arrow::write_parquet(data_object, full_path)
  } else {
    stop("Unsupported file type. Please use 'csv' or 'parquet'.")
  }

  # Log the export
  log_msg(paste0("Exported : ", full_path))
}

## 5.1. Routine without outliers: `dhis2_routine_outliers_removed`
As `XXX_routine_removed-outliers-<method_name>`.parquet/csv

In [ ]:
# Exporting removed data
export_data(dhis2_routine_outliers_removed, "_removed", "csv")
export_data(dhis2_routine_outliers_removed, "_removed", "parquet")

## 5.2. Routine with **imputed** values: `dhis2_routine_imputed`
As `XXX_routine_outliers-<method_name>_imputed`.parquet/csv

In [ ]:
# Exporting imputed data
export_data(dhis2_routine_imputed, "_imputed", "csv")
export_data(dhis2_routine_imputed, "_imputed", "parquet")

# 6. Write to Dataset

In [ ]:
# # Double check that {reticulate} finds the module ... 
# openhexa

## 6.1. Create new **dataset** (coded)
This step could be skipped if preference for creating the Dataset manually  ...

Check if a dataset with slug == "**dhis2-outliers-detection**" already exists, if not create one.<br>
This avoids silentily creating a new dataset with same name (yet different slug and id) at each run ...

In [ ]:
# # Define Dataset 
# DATASET_SLUG = "dhis2-outliers-removal-imputation" # "dhis2-outliers-detection"
# DATASET_NAME <- "DHIS2_OUTLIERS_REMOVAL_IMPUTATION" # "DHIS2_OUTLIERS_DETECTION"
# DATASET_DESCRIPTION <- "Routine DHIS2 data without outliers and (optional) with imputer values. Outliers removal follows flagging based on various methods - see dedicated notebook for mode details."


In [ ]:
# # List all datasets 
# existing_datasets <- openhexa$workspaces$workspace$list_datasets()

# # Check if a dataset with slug "dhis2-reporting-rate" exists (DATASET_SLUG)
# dataset_exists <- FALSE
# for (ds in existing_datasets) {
#   if (ds$slug == DATASET_SLUG) {
#     dataset_exists <- TRUE
#     dataset <- ds  # Store the existing dataset to be used downstream
#     break
#   }
# }

# # If it doesn't exist, create it 
# if (!dataset_exists) {
#   # Define name & description (modify as needed)
#   dataset_name <- DATASET_NAME
#   dataset_description <- DATASET_DESCRIPTION
  
#   # Create the dataset
#   dataset <- openhexa$workspaces$workspace$create_dataset(
#     name = dataset_name,
#     description = dataset_description
#   )
  
#   print(paste("Created new dataset:", dataset$slug))
# } else {
#   print(paste("Dataset already exists:", dataset$slug))
# }

## 6.2. Assign **version** to dataset - _if_ it does not have one yet
🚨 Note: brand-new datasets don't have a version yet! Version needs to be explicitly created! <br>
With this code a "**v0**" is created only if dataset doesn't have any (else it will keep its current version).

In [ ]:
# # Check if dataset has any versions
# if (is.null(dataset$latest_version)) {
#   print("Dataset has no versions. Creating initial version 'v0'...")
#   initial_version <- dataset$create_version("v0")
#   print(paste("Created version:", initial_version$name))
# } else {
#   print(paste("Dataset already has versions. Latest version:", dataset$latest_version$name))
# }

## 6.3. Define list of **files to be written** to the Dataset
Here we're writing _all_ the files that we previously exported as `.csv` and `.parquet` files to `/data/dhis2_incidence/`

In [ ]:
# EXPORTED_DATA_FOLDER = "dhis2_outliers_removal_imputation"

In [ ]:
# # Define the directory path (with expanded ~)
# dir_path <- path.expand(file.path(DATA_PATH, EXPORTED_DATA_FOLDER))

# # List all files in the directory
# file_list <- list.files(dir_path, full.names = TRUE, recursive = FALSE)

# # Check if there are any files
# if (length(file_list) == 0) {
#   stop("No files found in directory: ", dir_path)
# }

# # Print files to be uploaded (for debugging)
# print("Files to be uploaded:")
# print(file_list)

## 6.4. Create **new version** of Dataset and **write all files** to it
At each execution of the notebook (hence at each pipeline execution):
* **creates** a new version of the Dataset
* **writes** all files found in `file_list` to this (latest) Dataset version

In [ ]:
# # 1. Get latest dataset version name
# latest_version_name <- dataset$latest_version$name
# latest_version_nr <- as.numeric(gsub("v", "", latest_version_name))

# # 2. Create a new version (vX+1)
# new_version <- dataset$create_version(paste0("v", latest_version_nr + 1))

# # 3. Add all files to the new version
# for (file_path in file_list) {
#   # Extract just the filename (without full path)
#   filename <- basename(file_path)
  
#   # Upload the file
#   new_version$add_file(file_path, filename = filename)
  
#   # Print confirmation
#   print(paste("Uploaded:", filename))
# }

# print(paste("Successfully uploaded", length(file_list), "files to version", new_version$name))